In [1]:
!pip install -q transformers accelerate sentencepiece datasets


In [2]:
import pandas as pd
import torch
import json
import re
from transformers import pipeline
from tqdm import tqdm


In [3]:
df = pd.read_csv("/content/yelp.csv")

print(df.head())
print("Rows:", len(df))
print("Columns:", df.columns)


              business_id        date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg  2011-01-26  fWKvX83p0-ka4JS3dc6E5A      5   
1  ZRJwVLyzEJq1VAihDhYiow  2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      5   
2  6oRAC4uyJCsJl1X0WZpVSA  2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      4   
3  _1QQZuf4zZOyFCvXc0o6Vg  2010-05-27  G-WvGaISbqqaMHlNnByodA      5   
4  6ozycU1RpktNG2-1BroVtw  2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      5   

                                                text    type  \
0  My wife took me here on my birthday for breakf...  review   
1  I have no idea why some people give bad review...  review   
2  love the gyro plate. Rice is so good and I als...  review   
3  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4  General Manager Scott Petello is a good egg!!!...  review   

                  user_id  cool  useful  funny  
0  rLtl8ZkDX5vH5nAx9C3q5Q     2       5      0  
1  0a2KyEL0d3Yb1V6aivbIuQ     0       0      0  
2  0hT2KtfLiobPvh6cDC8JQg     0    

In [4]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device=-1   # force CPU (no crash)
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


In [12]:
def prompt_v1(review):
    return f"""
You must output ONLY a JSON object.

JSON schema:
{{"predicted_stars": 1-5, "explanation": "short"}}

Text:
{review}

JSON:
"""



In [13]:
def prompt_v2(review):
    return f"""
You must output ONLY a JSON object.

JSON schema:
{{"predicted_stars": 1-5, "explanation": "short"}}

Text:
{review}

JSON:
"""



In [14]:
def prompt_v3(review):
    return f"""
You must output ONLY a JSON object.

JSON schema:
{{"predicted_stars": 1-5, "explanation": "short"}}

Text:
{review}

JSON:
"""


In [15]:
def call_llm(prompt):
    output = generator(
        prompt,
        max_new_tokens=40,
        do_sample=False,
        eos_token_id=generator.tokenizer.eos_token_id
    )
    return output[0]["generated_text"]



In [16]:
def extract_json(text):
    if not text:
        return None

    # Try strict JSON first
    try:
        return json.loads(text.strip())
    except:
        pass

    # Fallback: find first JSON block
    match = re.search(r'\{[^{}]*\}', text)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None

    return None



In [17]:
def evaluate(prompt_fn, limit=200):
    correct = 0
    total = 0
    valid_json = 0

    for _, row in tqdm(df.head(limit).iterrows(), total=limit):
        raw_output = call_llm(prompt_fn(row["text"]))
        data = extract_json(raw_output)

        if data and "predicted_stars" in data:
            valid_json += 1

            predicted = int(data["predicted_stars"])
            actual = int(row["actual_stars"])

            if predicted == actual:
                correct += 1
        else:
            print("INVALID OUTPUT:", raw_output)

        total += 1

    return {
        "accuracy": correct / total,
        "json_validity": valid_json / total
    }


In [18]:
results_v1 = evaluate(prompt_v1)
results_v2 = evaluate(prompt_v2)
results_v3 = evaluate(prompt_v3)

results_v1, results_v2, results_v3


  0%|          | 1/200 [01:06<3:40:45, 66.56s/it]

INVALID OUTPUT: 
You must output ONLY a JSON object.

JSON schema:
{"predicted_stars": 1-5, "explanation": "short"}

Text:
My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait

  0%|          | 1/200 [02:20<7:46:23, 140.62s/it]


KeyboardInterrupt: 